# Learning to approximate an AC power flow

In this notebook, we explain how to use our package to train a simple neural network to imitate the output of an AC power flow simulator.

## Downloading a dataset

First of all, we need to download a dataset. We propose to download a small dataset of power grids derived from the case60nordic file (also known as nordic32), randomly generated using [powerdatagen](https://github.com/bdonon/powerdatagen).

The dataset is available on zenodo [here](https://zenodo.org/record/7077699). The following code downloads the dataset if it is not already here. Please be patient, as it may take several minutes (not more than 10 minutes though).

If you have already downloaded the dataset, then this does nothing.

In [1]:
%%bash
if [ ! -d data/case60/ ]
then
    zenodo_get '10.5281/zenodo.7077699' -o data/
    unzip -qq data/case60.zip -d data/
    rm data/case60.zip data/md5sums.txt
fi

## ??

In [2]:
from torch.utils.data import DataLoader
import numpy as np
import tqdm
import jax
import jax.numpy as jnp
from tqdm.notebook import tqdm

%load_ext autoreload
%autoreload 2

import sys; sys.path.insert(0, '../../..')
import ml4ps as mp

We need to import a backend, which will serve to read power grid data. In some more complex problem, it will be used to perform power grid simulations.

In [3]:
backend = mp.PandaPowerBackend()

In [16]:
train_dir = 'data/case60/train'

## Building a normalizer

In [ ]:
normalizer = mp.Normalizer(data_dir=train_dir, backend=backend, n_samples=1000, tqdm=tqdm)

## Building a train set and a data loader

The normalizer is fed to the data loader, so that ...

In [5]:
train_set = mp.PowerGridDataset(data_dir=train_dir, backend=backend, normalizer=normalizer)
train_loader = DataLoader(train_set,
                          batch_size=8,
                          shuffle=True,
                          num_workers=8,
                          collate_fn=mp.power_grid_collate,
                          prefetch_factor=8)

## Building a Fully Connected neural network

First of all, we need to tell the neural network which features it should take as input, and wich features we want it to output.

In [8]:
input_features = {'load': ['p_mw', 'q_mvar'], 'gen': ['p_mw', 'vm_pu'], 'ext_grid': ['vm_pu']}
output_features = {'bus': ['res_vm_pu']}

Since we are working with a fully connected neural network, we need to tell it how many object of each class will be present in the data. This is due to the fact that fully connected neural networks can only take vector data as input. By telling the neural network the amount of objects, it is able to initialize its weights.

In [11]:
a, x, nets = next(iter(train_loader))
n_obj = {k: np.max([np.shape(x_k_f)[1] for f, x_k_f in x[k].items()]) 
         for k in list(input_features.keys())+list(output_features.keys())}
print(n_obj)

{'load': 22, 'gen': 22, 'ext_grid': 1, 'bus': 60}


We may now initialize our neural network.

In [12]:
fully_connected = mp.FullyConnected(input_features=input_features,
                        output_features=output_features,
                        n_obj=n_obj,
                        hidden_dimensions=[1024,1024])

In addition, we need to specify post-processing functions.

In [13]:
functions = {'bus': {'res_vm_pu': [mp.AffineTransform(offset=1.)]}}
postprocessor = mp.PostProcessor(functions=functions)

## Training loop

In [14]:
from jax.example_libraries import optimizers

learning_rate = 3e-4
opt_init, opt_update, get_params = optimizers.adam(learning_rate)
opt_state = opt_init(fully_connected.weights)

In [15]:
def loss_function(params, x, y):
    y_hat = fully_connected.batch_forward(params, x)
    y_post = postprocessor(y_hat)
    loss = jnp.mean((y_post['bus']['res_vm_pu'] - y['bus']['res_vm_pu'])**2)
    return loss

@jax.jit
def update(params, x, y, opt_state, step):
    loss, grads = jax.value_and_grad(loss_function)(params, x, y)
    opt_state = opt_update(step, grads, opt_state)
    return get_params(opt_state), opt_state, loss

In [ ]:
step = 0
for epoch in range(5):
    for a, x, nets in (pbar := tqdm(train_loader)):
        step += 1
        y = backend.update_run_extract(nets, features={'bus':['res_vm_pu']})
        fully_connected.weights, opt_state, loss = update(fully_connected.weights, x, y, opt_state, step)
        pbar.set_description("Epoch {}, Loss = {:.2e}".format(epoch, loss))

  0%|          | 0/1250 [00:00<?, ?it/s]

  0%|          | 0/1250 [00:00<?, ?it/s]

## 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_losses)
plt.yscale('log')
plt.show()

In [ ]:
a, x, nets = next(iter(train_loader))

In [ ]:
# Perform prediction
#x_norm = normalizer(x)
y_hat = fully_connected.batch_forward(fully_connected.weights, x)
y_post = postprocessor(y_hat)
y_post = np.reshape(y_post['bus']['res_vm_pu'], [-1])

# Get ground truth
y_truth = backend.update_run_extract(nets, features={'bus':['res_vm_pu']})
y_truth = np.reshape(y_truth['bus']['res_vm_pu'], [-1])

# Compare results
plt.scatter(y_truth, y_post)
plt.xlabel('Ground truth')
plt.ylabel('Prediction')
plt.show()